# Keras Model

## Model 준비

### 시간 측정

In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.47 s


### 드라이브 마운트


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.21 ms


### directiory 설정


*   상황에 맞게 수정




In [ ]:
%cd /content/drive/My Drive/KISTI_TEAM/Image/Final_Imageset8_KSH

/content/drive/.shortcut-targets-by-id/10PuVModIagENTfncS7B4E2vWAppXX5Bo/KISTI_TEAM/Image/Final_Imageset8_KSH
time: 4.21 ms


### library 설치

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.3.0
time: 2.32 ms


## Model 설정



*   dataset은 beverage 폴더 안에 train, validation으로 나눠서 넣어야 합니다.





In [ ]:
#batch_size, img_height, img_width 설정
batch_size = 32
img_height = 224
img_width = 224

time: 1.19 ms


In [ ]:
#경로 설정
execution_path = os.getcwd()
beverage_path = os.path.join(execution_path, 'beverage')
train_dir = os.path.join(beverage_path, 'train')
validation_dir = os.path.join(beverage_path, 'validation')

time: 1.82 ms


In [ ]:
#데이터 셋 불러오기
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir, image_size=(img_height,img_width), batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(validation_dir, image_size=(img_height,img_width), batch_size=batch_size)

Found 13259 files belonging to 10 classes.
Found 3237 files belonging to 10 classes.
time: 766 ms


In [ ]:
#class 확인
class_names = train_ds.class_names
print(class_names)

['2프로 부족할 때', '데미소다 사과', '밀키스', '칠성 사이다', '코카콜라', '토레타', '파워 에이드', '펩시', '포카리 스웨트', '환타 오렌지']
time: 1.26 ms


## Model 학습



```
tf.keras.applications.InceptionResNetV2(
    include_top=True, weights=None, input_shape=(224,224,3),classes=10, classifier_activation='softmax')
```


```
tf.keras.applications.MobileNetV2(
  input_shape=(224,224,3), alpha=1.0, include_top=True, weights=None,
    input_tensor=None, pooling=None, classes=10, classifier_activation='softmax')
```





### 처음부터 학습하기

In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('./log.csv',append=True, separator=';')
callbacks = keras.callbacks.ModelCheckpoint("MobileNet_{epoch}.h5", monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), classes=10, weights=None, classifier_activation='softmax')
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

time: 1.28 s


In [ ]:
hist = model.fit(train_ds, validation_data = val_ds, batch_size=batch_size, epochs=1, callbacks=[csv_logger, callbacks]) #상황에 맞게 수정

309/415 [=====================>........] - ETA: 25:36 - loss: 2.0926 - accuracy: 0.3631

### 이어서 학습하기


In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('./log.csv',append=True, separator=';')
callbacks = keras.callbacks.ModelCheckpoint("MobileNet_{epoch}.h5", monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), weights=None,classes=10, classifier_activation='softmax')
model.load_weights(execution_path+'/'+'model'+'/'+'save_at_10.h5') #상황에 맞게 수정
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

time: 1.56 s


In [ ]:
hist = model.fit(train_ds, validation_data = val_ds, batch_size=batch_size, epochs=20, callbacks=[csv_logger,callbacks], initial_epoch=10) #상황에 맞게 수정

Epoch 11/20
 86/415 [=====>........................] - ETA: 1:34:44 - loss: 5.9624 - accuracy: 0.9495

KeyboardInterrupt: ignored

time: 27min 11s


In [ ]:
# 6 훈련 과정 시각화 (정확도)
plt.plot(history.hist['accuracy'])
plt.plot(history.hist['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# 7 훈련 과정 시각화 (손실)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Convert


### h5 to tflite convert


In [ ]:
new_model= tf.keras.models.load_model(filepath="MobileNet_at_10.h5")
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = tflite_converter.convert()
open("MobileNet_model.tflite", "wb").write(tflite_model)

OSError: ignored

time: 23.7 ms


### h5 to pb convert


In [ ]:
model = tf.keras.models.load_model("MobileNet_at_10.h5")
export_dir = os.getcwd()
model.save(export_dir, save_format="tf")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/.shortcut-targets-by-id/10PuVModIagENTfncS7B4E2vWAppXX5Bo/KISTI_TEAM/Image/Final_Imageset8/assets
time: 24.2 s


### pb to tflite convert


In [ ]:
saved_model_dir = os.getcwd()
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model= converter.convert()
open(saved_model_dir+'/model.tflite','wb').write(tflite_model)

14017156

time: 14.1 s


## 기타 참고할 내용

In [ ]:
hist = model.fit(X_train, Y_train, epochs=1000, batch_size=10, validation_data = (X_val, Y_val))

print(hist.history['loss'])
print(hist.history['acc'])
print(hist.history['val_loss'])
print(hist.history['val_acc'])